In [ ]:
import redis
import numpy as np
import pandas as pd
r = redis.Redis(host='localhost', port=6379, db=0)
dev = 'computer:ff:ff:ff:ff:ff:ff'
attr = 'mem_usage'
nrows = None

In [ ]:
cpu = np.array(r.json().get(dev, 'cpu_usage')).transpose(1, 0)
mem = np.array(r.json().get(dev, 'mem_usage')).transpose(1, 0)
net = np.array(r.json().get(dev, 'net_capture')).transpose(1, 0)
disk_read = np.array(r.json().get(dev, 'disk_read')).transpose(1, 0)
disk_write = np.array(r.json().get(dev, 'disk_write')).transpose(1, 0)

In [ ]:
data = np.array(r.json().get(dev, attr)[:nrows]) if nrows is not None else np.array(r.json().get(dev, attr))

In [ ]:
def load_data(dev, attr, nrows=None):
    data = np.array(r.json().get(dev, attr)[:nrows]) if nrows is not None else np.array(r.json().get(dev, attr))
    data = data.transpose((0,1))
    if attr.startswith('disk'):
        df = pd.DataFrame(data, columns=['timestamp', 'count', 'bytes', 'time'])
    else:
        df = pd.DataFrame(data, columns=['timestamp', attr])
    df['timestamp'] = pd.to_numeric(df['timestamp']).astype(np.int64)
    df = df.drop_duplicates(subset=['timestamp'])
    index_type = df.dtypes[df.columns[0]]
    df = df.set_index(df.columns[0])
    df.index = pd.to_datetime(df.index.values, unit="ms" if index_type in [np.int32, np.int64] else None)
    return df